In [15]:
from Fetcher import Dataset

# Load json dataset

In [43]:
import os
import json
import tzlocal
import numpy as np
import pandas as pd
from Fetcher import Dataset
from apscheduler.schedulers.blocking import BlockingScheduler
from apscheduler.events import EVENT_JOB_EXECUTED, EVENT_JOB_ERROR
from statsmodels.tsa.holtwinters import ExponentialSmoothing, SimpleExpSmoothing


list_town = ["Miami", "New York", "Las Vega", "Chicago", "Seattle", "San Francisco", "Washington","New Orleans", "Palm Springs", "San Diego", "Charleston"]
list_dataset = ["temp", "rainfall", "snowfall", "wind", "solar"]

d = Dataset()
final_df = []
for dataset in list_dataset[:1]:
    for town in list_town[:2]:
        print(f"dataset/{dataset}/{town}.json")
        if os.path.exists(f"dataset/{dataset}/{town}.json"):
            print(f"{dataset} - {town}")
            res = json.load(open(f"dataset/{dataset}/{town}.json"))
            data = res["data"]

            index = pd.to_datetime(list(data.keys()))
            values = [float(s) if s else None for s in data.values()]

            series = pd.Series(values, index=index)
            df = series.to_frame(name='Value')

            df = df[~df.index.astype(str).str.contains('02-29')]

            #algorithm
            print("Training...")
            hw_model = ExponentialSmoothing(df["Value"],
                      trend    ="add",
                      seasonal = "add",
                      seasonal_periods=365,
                      damped=False
                      ).fit(use_boxcox="log") # damped=False


            hw_fitted = hw_model.fittedvalues
            hw_resid = hw_model.resid
            days_in_future = 30
            # Adding the mean of the residuals to correct the bias.
            py_hw = hw_model.forecast(days_in_future) + np.mean(hw_resid)
            print(py_hw)
            
            # to frame
            df = py_hw.to_frame()

            # get lat and long
            lat, long = d.get_lat_lon(town)
            
            df["lat"] = str(lat)
            df["long"] = str(long)
            df["town"] = str(town)
            
            final_df.append(df)
            df = None

to_save = pd.concat(final_df)
to_save.to_pickle("temp_model.pkl")

dataset/temp/Miami.json
temp - Miami
Training...


/home/bm7/.local/lib/python3.7/site-packages/ipykernel_launcher.py:39: FutureWarning: the 'damped' keyword is deprecated, use 'damped_trend' instead
/home/bm7/.local/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:583: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/home/bm7/.local/lib/python3.7/site-packages/statsmodels/tsa/holtwinters/model.py:429: FutureWarning: After 0.13 initialization must be handled at model creation
  FutureWarning,
/home/bm7/.local/lib/python3.7/site-packages/statsmodels/tsa/holtwinters/model.py:1116: FutureWarning: Setting use_boxcox during fit has been deprecated and will be removed after 0.13. It must be set during model initialization.
  FutureWarning
/home/bm7/.local/lib/python3.7/site-packages/statsmodels/tsa/holtwinters/model.py:922: ConvergenceWarning: Optimization failed to converge. Check mle_ret

15004    79.600432
15005    80.032802
15006    81.829168
15007    81.286666
15008    80.110531
15009    79.756343
15010    80.276856
15011    81.976389
15012    81.214887
15013    81.145709
15014    81.169084
15015    82.539622
15016    83.194185
15017    83.354937
15018    83.841150
15019    83.657556
15020    83.024434
15021    82.728741
15022    82.836565
15023    82.244752
15024    82.422680
15025    82.274889
15026    82.413884
15027    82.222703
15028    82.287330
15029    82.207777
15030    82.933390
15031    83.147540
15032    80.932094
15033    81.316443
dtype: float64
dataset/temp/New York.json
temp - New York
Training...


/home/bm7/.local/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:583: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/home/bm7/.local/lib/python3.7/site-packages/statsmodels/tsa/holtwinters/model.py:922: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  ConvergenceWarning,
/home/bm7/.local/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:379: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  ValueWarning)


15004    79.600432
15005    80.032802
15006    81.829168
15007    81.286666
15008    80.110531
15009    79.756343
15010    80.276856
15011    81.976389
15012    81.214887
15013    81.145709
15014    81.169084
15015    82.539622
15016    83.194185
15017    83.354937
15018    83.841150
15019    83.657556
15020    83.024434
15021    82.728741
15022    82.836565
15023    82.244752
15024    82.422680
15025    82.274889
15026    82.413884
15027    82.222703
15028    82.287330
15029    82.207777
15030    82.933390
15031    83.147540
15032    80.932094
15033    81.316443
dtype: float64


In [41]:
df_open = pd.read_pickle("temp_model.pkl")

In [42]:
df_open

,0,lat,long,town
15004,79.600432,25.762329613614614,-80.19114735100034,Miami
15005,80.032802,25.762329613614614,-80.19114735100034,Miami
15006,81.829168,25.762329613614614,-80.19114735100034,Miami
15007,81.286666,25.762329613614614,-80.19114735100034,Miami
15008,80.110531,25.762329613614614,-80.19114735100034,Miami
15009,79.756343,25.762329613614614,-80.19114735100034,Miami
15010,80.276856,25.762329613614614,-80.19114735100034,Miami
15011,81.976389,25.762329613614614,-80.19114735100034,Miami
15012,81.214887,25.762329613614614,-80.19114735100034,Miami
15013,81.145709,25.762329613614614,-80.19114735100034,Miami
